In [34]:
import pickle
from src.models.ssl_models_curlie import embedding_classifier
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, hamming_loss, precision_score, recall_score,accuracy_score
# from torch import cdist
from scipy.spatial.distance import cdist
import torch
from torch import nn
import torch.nn.functional as F
from src.models.ssl_models_curlie.embedding_classifier import EmbeddingClassifier
from torch.utils.data import TensorDataset, DataLoader
import os
from torch import nn, optim

from dotenv import find_dotenv, load_dotenv
import logging
import json

In [11]:
filepath="../data_curlie/processed/selected_indices_sampled.pkl"
try:
    with open(filepath, 'rb') as file:
        selected_indices_sampled = pickle.load(file)
except Exception as e:
    raise

In [12]:
selected_indices_sampled

array([   387,    158,   1023,   1420,      4,   1181,   9167,    149,
          215,   1426,   3143,   6009,   1274,   4069,   5117,    207,
          145,    440,   8922,   1362,    136,   6622,    772,  11855,
          193,    186,     36,    444,    237,     47,   1130,     13,
          389,     44,    719,     24,    386,     74,    416, 528961,
        29078,   6616,   5230,      6,   2645,    583,    574,     62,
        90431,   1179,    124,    708,    621,     61,   2362,      1,
          654,      8,    506,  17438,    458,    398,     19,   1470,
           34,   4108,     75,    477,    348,    994,   5569,    694,
          879,    577,      0,    159,    210,   1158,    261,   2920,
         6032,   1444,    585,   8099,   7131,    904,    457,    196,
           16,    119,     11,    461,    181,      3,   2268,    374,
        24871,   4722,     64,    625,     33,     14,    135,     45,
         1631,   9271,    162,   3629,    763,     22,    412,   1265,
      

In [13]:
len(selected_indices_sampled)

300

In [18]:
filepath="../data_curlie/processed/train_sampled.pkl"
try:
    with open(filepath, 'rb') as file:
        train_sampled = pickle.load(file)
except Exception as e:
    raise

In [37]:
train_sampled

,uid,header_embeddings,url,h2v_pred,h2v_embedding,label,dataset
0,143215,"[-0.12137477099895477, 0.3743447065353393, 0.0...",www.sg-adelsberg.de,"[0.0006075641140341759, 0.003006806131452322, ...","[-2.8622231483459473, 1.2275725603103638, 1.04...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",train
1,657885,"[-0.04726717248558998, 0.0948432385921478, 0.1...",www.myguitarsolo.com,"[0.9886903166770935, 0.011595186777412891, 0.0...","[1.731790542602539, 2.1384191513061523, -1.021...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",train
2,973532,"[-0.01860016956925392, 0.3559060394763946, 0.2...",www.factoryfive.com,"[0.014509319327771664, 0.6098007559776306, 0.0...","[-0.6853832602500916, 0.21162883937358856, 0.3...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",train
3,1112170,"[-0.044978976249694824, 0.2400597184896469, 0....",www.socalkees.org,"[9.662981028668582e-07, 0.003199808532372117, ...","[-2.572124719619751, 0.4726077616214752, 3.399...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",train
4,986375,"[-0.0047127907164394855, 0.1424452066421508, 0...",www.vordingborgswim.dk,"[0.008630601689219475, 0.013073411770164967, 0...","[-2.650256872177124, 0.7773601412773132, 1.352...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",train
...,...,...,...,...,...,...,...
10995,1975649,"[0.19816362857818604, 0.2595343589782715, -0.1...",unimex-bg.com,"[0.3214002549648285, 0.7721798419952393, 0.088...","[0.9274794459342957, 0.14115266501903534, -0.2...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",train
10996,1019643,"[0.06207523122429848, 0.2220538854598999, -0.0...",www.web.de,"[0.1325898915529251, 0.16243311762809753, 0.97...","[1.8066908121109009, 1.5915579795837402, -1.12...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",train
10997,1458708,"[0.013551934622228146, -0.3369513154029846, 0....",www.cetir.com,"[0.02369690127670765, 0.3997385501861572, 0.08...","[1.3136509656906128, -1.272384762763977, 0.524...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",train
10998,613223,"[0.032556768506765366, 0.0402247346937656, 0.0...",www.wouf.net,"[0.1965414583683014, 0.09160946309566498, 0.03...","[-0.19722652435302734, 1.7930998802185059, 1.6...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",train


In [26]:
embeddings=train_sampled['header_embeddings']
embeddings

0        [-0.12137477099895477, 0.3743447065353393, 0.0...
1        [-0.04726717248558998, 0.0948432385921478, 0.1...
2        [-0.01860016956925392, 0.3559060394763946, 0.2...
3        [-0.044978976249694824, 0.2400597184896469, 0....
4        [-0.0047127907164394855, 0.1424452066421508, 0...
                               ...                        
10995    [0.19816362857818604, 0.2595343589782715, -0.1...
10996    [0.06207523122429848, 0.2220538854598999, -0.0...
10997    [0.013551934622228146, -0.3369513154029846, 0....
10998    [0.032556768506765366, 0.0402247346937656, 0.0...
10999    [-0.1407049000263214, 0.0497642941772937, 0.21...
Name: header_embeddings, Length: 10000, dtype: object

In [25]:

def get_device():
    # Set random seed for reproducibility
    torch.manual_seed(0)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(0)
        device = 'cuda'
    else:
        device = 'cpu'
    print("Device: ", device)
    return device
    
    

In [52]:
print("Training the USL SSL model...")

device = get_device()
labels=train_sampled['label']
input_dim = 767
print("Input dimension: ", input_dim)
num_classes = 14  # Assuming labels are one-hot encoded
print("Number of classes: ", num_classes)


# Assuming EmbeddingClassifier is a defined model suitable for your needs
model = EmbeddingClassifier(input_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Analyzing class imbalance
labels=np.array(labels.tolist())
df_labels = pd.DataFrame(labels, columns=[f"label{i}" for i in range(labels.shape[1])])
class_frequencies = df_labels.mean()  # Proportion of positive instances for each label
print("\nClass Frequencies:\n", class_frequencies)

# Assuming 'class_frequencies' is a Pandas Series from your previous message
weights = (1 / class_frequencies) / (1 / class_frequencies).sum()
pos_weight = torch.tensor(weights.values).float().to(device)

# Update your loss function to use pos_weight
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


# Preparing DataLoaders
labeled_embeddings = embeddings
labeled_labels = labels  # Directly use labels without converting to numpy array if already tensor


# Convert to tensors and create datasets
labeled_dataset = TensorDataset(torch.tensor(labeled_embeddings, dtype=torch.float32),
                                torch.tensor(labeled_labels, dtype=torch.float32))


# DataLoaders
labeled_loader = DataLoader(labeled_dataset, batch_size=64, shuffle=True)


# Training loop
for epoch in range(50):  # Number of epochs
    model.train()
    total_loss = 0
    for data, target in labeled_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(labeled_loader)}')

    # Implementing a strategy for unlabeled data if needed
    # This could involve pseudo-labeling, consistency regularization, etc.

# You could add validation steps, model saving, etc., as needed.


Training the USL SSL model...
Device:  cuda
Input dimension:  767
Number of classes:  14

Class Frequencies:
 label0     0.0956
label1     0.2836
label2     0.0611
label3     0.0183
label4     0.0623
label5     0.0141
label6     0.0115
label7     0.0112
label8     0.0869
label9     0.0446
label10    0.0460
label11    0.0797
label12    0.1399
label13    0.0677
dtype: float64
Epoch 1, Loss: 0.05223982025436155
Epoch 2, Loss: 0.01634586414998504
Epoch 3, Loss: 0.016176146787300612
Epoch 4, Loss: 0.016098297076525204
Epoch 5, Loss: 0.01602807333159029
Epoch 6, Loss: 0.015969539547611954
Epoch 7, Loss: 0.015950796413858225
Epoch 8, Loss: 0.015858007689854903
Epoch 9, Loss: 0.015774582103369343
Epoch 10, Loss: 0.01576668164058096
Epoch 11, Loss: 0.01561252541460429
Epoch 12, Loss: 0.015528230138929787
Epoch 13, Loss: 0.015382328563055415
Epoch 14, Loss: 0.015226261215699706
Epoch 15, Loss: 0.01511301648963219
Epoch 16, Loss: 0.014930134639143944
Epoch 17, Loss: 0.01478028913164974
Epoch 18, 

In [53]:
filepath="../data_curlie/processed/val_sampled.pkl"
try:
    with open(filepath, 'rb') as file:
        val_sampled = pickle.load(file)
except Exception as e:
    raise

In [54]:
labels_val=val_sampled['label'].tolist()
embeddings_val=val_sampled['header_embeddings'].tolist()

In [56]:
print("Evaluating the USL SSL model...:  ")
device=get_device()

# embeddings_val = min_max_scale_embeddings(embeddings_val)#normalize embeddings

# Load the true labels
val_labels = np.array(labels_val)
print("True labels: ",val_labels)


# Analyzing class imbalance
df_labels = pd.DataFrame(val_labels, columns=[f"label{i}" for i in range(val_labels.shape[1])])
class_frequencies = df_labels.mean()  # Proportion of positive instances for each label
print("\nClass Frequencies:\n", class_frequencies)


# Predictions from the SSL model
val_embeddings_tensor = torch.tensor(np.array(embeddings_val), dtype=torch.float32).to(device)

threshold=0.5
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Obtain model predictions (logits)
    val_outputs = model(val_embeddings_tensor)
    # Convert logits to probabilities
    probabilities = torch.sigmoid(val_outputs)
    print("Probabilities:")
    print(probabilities)
    # Apply threshold to get binary predictions for each class
    predictions = (probabilities > threshold).int().cpu().numpy()
    predictions
    
print("Predictions from the SSL model: ",predictions)

# Evaluate SSL-enhanced model
hamming_loss_ssl = hamming_loss(val_labels, predictions)
precision_ssl = precision_score(val_labels,predictions, average='micro')
recall_ssl = recall_score(val_labels, predictions, average='micro')
f1_ssl = f1_score(val_labels, predictions, average='micro')

# Print results
print("Validation Results:")
print(f"SSL Model - Hamming Loss: {hamming_loss_ssl}, Precision: {precision_ssl}, Recall: {recall_ssl}, F1 Score: {f1_ssl}")


Evaluating the USL SSL model...:  
Device:  cuda
True labels:  [[0 0 0 ... 0 1 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]]

Class Frequencies:
 label0     0.108
label1     0.258
label2     0.072
label3     0.014
label4     0.046
label5     0.022
label6     0.006
label7     0.008
label8     0.064
label9     0.052
label10    0.034
label11    0.094
label12    0.152
label13    0.084
dtype: float64
Probabilities:
tensor([[3.3069e-03, 2.7587e-03, 4.5342e-05,  ..., 2.0535e-04, 1.2115e-03,
         7.9171e-03],
        [9.6331e-04, 1.2865e-03, 3.0796e-06,  ..., 6.4664e-04, 1.4372e-03,
         2.5112e-03],
        [1.7975e-03, 2.7533e-03, 5.4590e-05,  ..., 3.9604e-04, 1.7014e-03,
         3.2003e-04],
        ...,
        [2.9790e-03, 1.6150e-04, 7.7408e-03,  ..., 2.2524e-04, 2.0458e-04,
         1.3685e-03],
        [1.7911e-04, 7.7685e-04, 6.5966e-03,  ..., 1.4853e-04, 1.0234e-03,
         8.8540e-04],
        [2.6876e-03, 5.8213e-0